# Deep Learning

**ARCATS Workshop**

Welcome! In this notebook, you'll learn the basics of deep learning with **PyTorch**. We’ll cover:
- Tensors
- Neural networks
- Training on simple datasets

## 1. Introduction
Deep learning is a type of machine learning that uses **neural networks** to learn from data. PyTorch is a popular library for building and training these models.

## 2. Tensors in PyTorch
Tensors are like NumPy arrays, but optimized for GPUs.

In [1]:
import torch

In [2]:
# Create a tensor
x = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
print("Tensor x:\n", x)

Tensor x:
 tensor([[1., 2.],
        [3., 4.]])


In [3]:
# Basic operations
print("x + x =\n", x + x)
print("x * 2 =\n", x * 2)

x + x =
 tensor([[2., 4.],
        [6., 8.]])
x * 2 =
 tensor([[2., 4.],
        [6., 8.]])


### Exercise 1

Create a 3x3 tensor of random numbers with `torch.rand` and multiply it by 5.

## 3. Building a Simple Neural Network
We’ll use a small neural network to classify points in the **Iris dataset**.

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.optim as optim

In [5]:
# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

In [6]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
# Convert to tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [8]:
# Define model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 3)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
model = Net()

In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:
# Training loop
for epoch in range(50):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

print("Final training loss:", loss.item())

Final training loss: 0.17436008155345917


### Exercise 2

Try changing the number of hidden units in `fc1` (e.g., 32 instead of 16). Does accuracy improve?

## 4. Evaluating the Model

In [12]:
with torch.no_grad():
    test_outputs = model(X_test)
    predictions = torch.argmax(test_outputs, dim=1)
    accuracy = (predictions == y_test).float().mean()
    print("Test Accuracy:", accuracy.item())


Test Accuracy: 1.0


### Exercise 3

Change the optimizer from `Adam` to `SGD` and see how performance changes.

## 5. Deep Learning on Images (MNIST)
Now let’s use a neural network on the **MNIST dataset** of handwritten digits.

In [13]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [14]:
# Transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [15]:
# Load dataset
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

100.0%
100.0%
100.0%
100.0%


In [16]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000, shuffle=False)

In [17]:
# Define model
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
mnist_model = MNISTNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mnist_model.parameters(), lr=0.001)

In [19]:
num_epochs = 1  # number of epochs

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()           # reset gradients
        outputs = mnist_model(images)  # forward pass
        loss = criterion(outputs, labels)  # compute loss
        loss.backward()                 # backpropagate
        optimizer.step()                # update weights

        running_loss += loss.item()     # accumulate loss

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/1], Loss: 0.3861


In [20]:
mnist_model.eval()  # set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # no need to compute gradients
    for images, labels in test_loader:
        outputs = mnist_model(images)
        _, predicted = torch.max(outputs.data, 1)  # get class with highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 94.03%


### Exercise 4

Change the number of epochs `num_epochs` and train for several epochs. How does accuracy improve?

## 5. Convolutional Neural Networks (CNNs)

In [21]:
class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # after 2 pooling layers, 28x28 -> 7x7
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [22]:
# Instantiate model, loss, optimizer
mnist_cnn = MNISTCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mnist_cnn.parameters(), lr=0.001)

In [23]:
# Training loop (1 epoch for example)
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = mnist_cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/1], Loss: 0.1597


In [24]:
# Test accuracy
mnist_cnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = mnist_cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 98.46%


## 6. Summary
You learned:
- Tensors in PyTorch
- Building and training a simple neural network
- Evaluating performance
- Training a neural net on images (MNIST) with Fully connected Neural Network
- Use Convolutional Neural Networks (CNNs) for better image accuracy